In [1]:
from py2neo import Graph

In [2]:
from searchapp.models import *

In [3]:
graph = Graph("bolt://neo4j:7687", auth=("neo4j", "123qweqwe"))

In [4]:
ids = ['MONDO_5515_oral_cavity_cancer', 'CTD_C024746_tobacco_tar']

In [5]:
hop = 5

In [6]:
query = graph.query(f"""
        MATCH (n)
        RETURN n
        LIMIT 10;
    """)

In [7]:
data = query.data()
data

[{'n': Node('Gene/Protein', id='NCBI_9796_PHYHIP', name='phyhip', original_name='PHYHIP', source='NCBI')},
 {'n': Node('Gene/Protein', id='NCBI_7918_GPANK1', name='gpank1', original_name='GPANK1', source='NCBI')},
 {'n': Node('Gene/Protein', id='NCBI_8233_ZRSR2', name='zrsr2', original_name='ZRSR2', source='NCBI')},
 {'n': Node('Gene/Protein', id='NCBI_4899_NRF1', name='nrf1', original_name='NRF1', source='NCBI')},
 {'n': Node('Gene/Protein', id='NCBI_5297_PI4KA', name='pi4ka', original_name='PI4KA', source='NCBI')},
 {'n': Node('Gene/Protein', id='NCBI_6564_SLC15A1', name='slc15a1', original_name='SLC15A1', source='NCBI')},
 {'n': Node('Gene/Protein', id='NCBI_8668_EIF3I', name='eif3i', original_name='EIF3I', source='NCBI')},
 {'n': Node('Gene/Protein', id='NCBI_10826_FAXDC2', name='faxdc2', original_name='FAXDC2', source='NCBI')},
 {'n': Node('Gene/Protein', id='NCBI_4489_MT1A', name='mt1a', original_name='MT1A', source='NCBI')},
 {'n': Node('Gene/Protein', id='NCBI_6272_SORT1', name

In [8]:
query = graph.query(f"""
        MATCH (n) where n.id IN ["{'","'.join(ids)}"]
        WITH collect(n) as nodes
        UNWIND nodes as n
        UNWIND nodes as m
        WITH * WHERE id(n) < id(m)
        MATCH path = shortestpath( (n)-[r *..{hop}]-(m))
        RETURN path
        LIMIT 10;
    """)

In [9]:
data = query.data()
data

[{'path': Path(Node('Exposure', id='CTD_C024746_tobacco_tar', name='tobacco tar', original_name='tobacco tar', source='CTD'), Exposure_Disease(Node('Disease', id='MONDO_grouped_3781_5607_18597_bronchitis', name='bronchitis', original_name='bronchitis', source='MONDO_grouped'), Node('Exposure', id='CTD_C024746_tobacco_tar', name='tobacco tar', original_name='tobacco tar', source='CTD'), name='linked to'), Disease_Protein(Node('Gene/Protein', id='NCBI_7422_VEGFA', name='vegfa', original_name='VEGFA', source='NCBI'), Node('Disease', id='MONDO_grouped_3781_5607_18597_bronchitis', name='bronchitis', original_name='bronchitis', source='MONDO_grouped'), name='associated with'), Disease_Protein(Node('Gene/Protein', id='NCBI_7422_VEGFA', name='vegfa', original_name='VEGFA', source='NCBI'), Node('Disease', id='MONDO_5627_head_and_neck_cancer', name='head and neck cancer', original_name='head and neck cancer', source='MONDO'), name='associated with'), Disease_Disease(Node('Disease', id='MONDO_551

In [11]:
labels_qs = Label.objects.all()

In [12]:
for label in labels_qs:
        print(label.name)

Gene/Protein
Drug
Effect/Phenotype
Disease
Biological_Process
Molecular_Function
Cellular_Component
Exposure
Pathway
Anatomy
